# Deep Q Network Agent on Cassava Leaf Disease Imbalanced Dataset

In this notebook, we are going to apply our custom created DQN Agent, on Casssava Leaf Disease Imabalnced Dataset and will try to get abetter accuracy on the classification for multi-class imbalanced classification

In [3]:
# Importing Libraries
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from collections import deque
import random
from qnetwork import DQNetwork
from dataset import Cifar10ImageDataset, CassavaLeafDataset, PersonalityDataset
from agent import DQNAgent

In [4]:
dataset = PersonalityDataset()

       Response Id  You regularly make new friends.  \
0            False                            False   
1            False                            False   
2            False                            False   
3            False                            False   
4            False                            False   
...            ...                              ...   
59994        False                            False   
59995        False                            False   
59996        False                            False   
59997        False                            False   
59998        False                            False   

       You spend a lot of your free time exploring various random topics that pique your interest  \
0                                                  False                                            
1                                                  False                                            
2                                   

Label is  ['ENFJ' 'ENFP' 'ENTJ' 'ENTP' 'ESFJ' 'ESFP' 'ESTJ' 'ESTP' 'INFJ' 'INFP'
 'INTJ' 'INTP' 'ISFJ' 'ISFP' 'ISTJ' 'ISTP']
Labels with count {'ENFJ': 3743, 'ENFP': 3760, 'ENTJ': 3737, 'ENTP': 3760, 'ESFJ': 3746, 'ESFP': 3769, 'ESTJ': 3759, 'ESTP': 3749, 'INFJ': 3761, 'INFP': 3734, 'INTJ': 3743, 'INTP': 3742, 'ISFJ': 3739, 'ISFP': 3746, 'ISTJ': 3756, 'ISTP': 3755}


In [5]:
dataset.y_train.shape

(39001,)

In [6]:
dataset.X_train.shape

(39001, 61)

In [ ]:
network = DQNetwork(state_size=(dataset.X.shape[1],), action_size=(len(np.unique(dataset.y))), learning_rate = 1e-4, image=False)

In [8]:
"HAHAHA"

'HAHAHA'

In [ ]:
agent = DQNAgent(network, dataset, state_size=(dataset.X.shape[1],), action_size=(len(np.unique(dataset.y))), memory=deque(maxlen=2000), gamma = 0.95,epsilon=1.0)
agent.train_personality(10, 20)

(61,)
Action is 14 and label is 4
Reward is -0.08128
(61,)
Action is 2 and label is 11
Reward is -0.080684
(61,)
Action is 5 and label is 15
Reward is -0.079832
(61,)
Action is 9 and label is 6
Reward is -0.080096
(61,)
Action is 10 and label is 3
Reward is -0.200739
(61,)
Action is 2 and label is 5
Reward is -0.081199
(61,)
Action is 14 and label is 14
Reward is 0.080202
(61,)
Action is 13 and label is 4
Reward is -0.08128
(61,)
Action is 9 and label is 10
Reward is -0.079465
(61,)
Action is 6 and label is 15
Reward is -0.079832
(61,)
Action is 0 and label is 8
Reward is -0.081008
(61,)
Action is 1 and label is 15
Reward is -0.079832
(61,)
Action is 13 and label is 14
Reward is -0.080202
(61,)
Action is 2 and label is 10
Reward is -0.079465
(61,)
Action is 10 and label is 1
Reward is -0.404499
(61,)
Action is 7 and label is 14
Reward is -0.080202
(61,)
Action is 6 and label is 9
Reward is -0.081035
(61,)
Action is 3 and label is 14
Reward is -0.080202
(61,)
Action is 7 and label is 5


In [ ]:
# Testing the model
total_reward = 0
train_labels = []
train_predictions = []
test_labels = []
test_predictions = []

for index, state in enumerate(agent.dataset.X_train):
    label = agent.dataset.y_train[index]
    state = np.reshape(state, [-1, agent.state_size[0]])
    action = agent.act(state)
    # action = np.argmax(self.network.model.predict(image), axis=1)
    reward, terminal = agent.get_reward_and_terminal(label, action)
    total_reward += reward
    train_labels.append(label)
    train_predictions.append(action)

In [ ]:
from sklearn import metrics

In [ ]:
metrics.accuracy_score(train_labels, train_predictions) * 100

In [ ]:
agent.evaluate_personality()

In [ ]:
"HAHAH"

In [ ]:
image_size = (256, 256)
channels = 3
batch_size = 32
action_size = 5

gamma = 0.95
epsilon = 1.0
learning_rate = 1e-4

episodes = 100

In [ ]:
dataset = Cifar10ImageDataset(20)
        
network = DQNetwork(state_size=(32, 32, 3), action_size=10, learning_rate = 1e-4)

agent = DQNAgent(network, dataset, state_size=(32, 32, 3), action_size = 10, memory=deque(maxlen=2000), gamma = 0.95,epsilon=1.0)
agent.train_cifar10(50)
# agent.save_model("./model/cifar10_model.h5")


In [ ]:
agent.evaluate_cifar10()

In [ ]:
dataset = CassavaLeafDataset(image_size=image_size, batch_size=batch_size)

In [ ]:
network = DQNetwork(state_size=(image_size[0], image_size[1], channels), action_size=action_size, learning_rate = learning_rate)

In [ ]:
agent = DQNAgent(network, dataset, state_size=image_size + (channels), action_size=action_size, memory=deque(maxlen=2000), gamma=gamma, epsilon=epsilon)

In [ ]:
agent.train(episodes)

In [ ]:
agent.evaluate()  